<a href="https://colab.research.google.com/github/O-Aiden/CartoonGuidetoStatisticsByPython/blob/main/15%E3%80%8A%E6%BC%AB%E7%94%BB%E7%BB%9F%E8%AE%A1%E5%AD%A6%E4%B9%8B%E5%9B%A0%E5%AD%90%E5%88%86%E6%9E%90%E3%80%8Bpython%E5%AE%9E%E7%8E%B0_%E4%B8%BB%E6%88%90%E5%88%86%E5%88%86%E6%9E%90%E7%9A%84%E6%B5%81%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import pandas as pd
import numpy as np
data = pd.read_excel('人气拉面馆评价数据.xlsx',index_col=0)
data.head(5)

,面,配料,汤
菜名,,,
二乐,2,4,5
梦田屋,1,5,1
地回,5,3,4
菜之花,2,2,3
花之节,3,5,5


# 求出主成分和主成分得分

## 步骤1：变量标准化

### 非标准化分析

In [79]:
# 求平均值
temp1_mean = pd.DataFrame(data.mean())
temp1_mean.columns = ['平均值']
temp1_mean = temp1_mean.T
temp1 = pd.concat([data.copy(),temp1_mean])
## 求标准差：在主成分分析的标准化过程中，标准差的分母通常为"数据个数-1"
# 自定义函数求主成分分析标准差
def pca_std(columns):
  sum_a = 0
  for i in range(len(data[columns])):
    a = np.square(data[columns][i] - temp1_mean[columns]['平均值'])
    sum_a = sum_a + a

    if i==len(data[columns])-1:
      std_a = round(np.sqrt(sum_a/(len(data[columns])-1)) ,1)
      
  return(std_a)
# 获取标准差
temp1_std = []
for column in list(data.columns.values):
  temp1_std.append(pca_std(column)) 
# 合并数据
temp1_std = pd.DataFrame(temp1_std)
temp1_std.columns = ['标准差']
temp1_std = temp1_std.T
temp1_std.columns = ['面','配料','汤']
temp1 = pd.concat([temp1,temp1_std])
temp1

,面,配料,汤
二乐,2.0,4.0,5.0
梦田屋,1.0,5.0,1.0
地回,5.0,3.0,4.0
菜之花,2.0,2.0,3.0
花之节,3.0,5.0,5.0
升辰轩,4.0,3.0,2.0
丸藏拉面,4.0,4.0,3.0
海乐亭,1.0,2.0,1.0
鸣海家,3.0,3.0,2.0
奏月,5.0,5.0,3.0


### 标准化分析

In [166]:
# 标准化过程：（每一数据 - 平均值）／标准差
temp2 = data.copy()
temp2['"面"的标准值u1'] = round((temp2['面']-temp1['面']['平均值'])/temp1['面']['标准差'],1)
temp2['"配料"的标准值u2'] = round((temp2['配料']-temp1['配料']['平均值'])/temp1['配料']['标准差'],1)
temp2['"汤"的标准值u3'] = round((temp2['汤']-temp1['汤']['平均值'])/temp1['汤']['标准差'],1)
temp2 = temp2.drop(['面','配料','汤'], axis = 1)

# 添加平均值和标准差
temp2 = temp2.append(pd.DataFrame([[0,0,0], [1,1,1]], columns=['"面"的标准值u1','"配料"的标准值u2','"汤"的标准值u3'], index = ['平均值','标准差']))
temp2

,"""面""的标准值u1","""配料""的标准值u2","""汤""的标准值u3"
二乐,-0.7,0.3,1.5
梦田屋,-1.3,1.2,-1.4
地回,1.3,-0.5,0.8
菜之花,-0.7,-1.3,0.1
花之节,0.0,1.2,1.5
升辰轩,0.7,-0.5,-0.6
丸藏拉面,0.7,0.3,0.1
海乐亭,-1.3,-1.3,-1.4
鸣海家,0.0,-0.5,-0.6
奏月,1.3,1.2,0.1


## 步骤2：求相关矩阵

In [125]:
temp3 = data.copy()
temp3_corr = round(temp3.corr(),2)
temp3_corr = np.array(temp3_corr)

## 步骤3:求特征值和特征向量

In [126]:
"""可能由于Python版本的问题，这里计算出来的最大的特征值对应的特征向量的符号与书中不符合"""
w, v = np.linalg.eig(temp3_corr)
# 其中特征值为w,特征向量为v
w = w.round(1)
v = v.round(2)
print('特征值: ', w)
print('特征向量: ', v)


特征值:  [1.6 0.8 0.6]
特征向量:  [[-0.57 -0.6  -0.56]
 [-0.52  0.79 -0.32]
 [-0.63 -0.11  0.77]]


## 步骤4:画出散点图
取最大的特征值对应的特征向量和第2大的特征值对应的特征向量

In [141]:
# 删除点3大特征值对应的特征向量
matrix_coordinate = np.delete(v, -1, axis=1)
matrix_coordinate
matrix_coordinate = np.array([[0.57, -0.6 ],
       [0.52,  0.79],
       [0.63, -0.11]])

In [142]:
df_coordinate = pd.DataFrame(matrix_coordinate,columns=['x','y'],index=['面','配料','汤'])
df_coordinate

,x,y
面,0.57,-0.60
配料,0.52,0.79
汤,0.63,-0.11


In [147]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(x=df_coordinate['x'], y=df_coordinate['y'],text=df_coordinate.index)
fig.show()

## 步骤5:得出第1主成分和第二主成分
最大的特征值对应的特征向量就是第一大主成分的系数。同样，第n大特征值对应的特征向量就是第n大主成分的系数。

In [149]:
pca_matrix = matrix_coordinate.T
pca_matrix

array([[ 0.57,  0.52,  0.63],
       [-0.6 ,  0.79, -0.11]])

## 步骤6:求出个样本在第1主成分和第2主成分中的坐标

In [171]:
temp2['第一主成分z1'] = (temp2['"面"的标准值u1']*0.57 + temp2['"配料"的标准值u2']*0.52 + temp2['"汤"的标准值u3']*0.63).round(2)
temp2['第二主成分z2'] = (temp2['"面"的标准值u1']*(-0.6) + temp2['"配料"的标准值u2']*(0.79) + temp2['"汤"的标准值u3']*(-0.11)).round(2)
temp2_copy = temp2.copy()
pca_temp2 = temp2_copy.drop(labels=['平均值','标准差'],axis=0)
pca_temp2 = pca_temp2.drop(labels=['"面"的标准值u1','"配料"的标准值u2','"汤"的标准值u3'],axis=1) 
pca_temp2

,第一主成分z1,第二主成分z2
二乐,0.70,0.49
梦田屋,-1.00,1.88
地回,0.98,-1.26
菜之花,-1.01,-0.62
花之节,1.57,0.78
升辰轩,-0.24,-0.75
丸藏拉面,0.62,-0.19
海乐亭,-2.30,-0.09
鸣海家,-0.64,-0.33
奏月,1.43,0.16


## 步骤七：画出散点图

In [173]:
import plotly.express as px
fig = px.scatter(x=pca_temp2['第一主成分z1'], y=pca_temp2['第二主成分z2'],text=pca_temp2.index)
fig.show()

# 直接利用sklearn封装的PCA进行主成分分析

In [185]:
"""可能由于Python版本的问题，这里计算出来的最大的特征值对应的特征向量的符号与书中不符合"""

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import pandas as pd
import numpy as np


df = data.copy()
data_stan = scale(df.values) # 标准化，标准化之后就自动根据协方差矩阵进行主成分分析了
# data2 = np.corrcoef(np.transpose(data)) # 没有必要单独计算协方差阵或相关系数阵
pca = PCA(n_components = 2) # 可以调整主成分个数，n_components = 1
pca.fit(data_stan)
print("特征值：",pca.explained_variance_) # 输出特征根
print("特征向量：",pca.components_) # 输出主成分

特征值： [1.7476154  0.90445369]
特征向量： [[-0.571511   -0.52211611 -0.63306393]
 [-0.60447096  0.78960694 -0.105526  ]]
